<p align="left">
  <img src="./logo_UTN.svg" />
</p>


# **Teoría de Circuitos II - R4001 - 2023**
## Trabajo Semanal 2
### Autor: Bruno Glecer

## Consigna

La tarea semanal consiste en realizar un analisis del siguiente circuito

<div>
    <img src="circuito.png" width="1000"/>
</div>

Se piden las siguientes consignas:

1. Hallar la transferencia $T = \frac{V_o}{V_i}$ en función de $\omega_0$ y $Q$
2. Obtener el valor de los componentes del circuito de forma tal que $\omega_0 = 1$ y $Q = 3$
3. Ajustar el valor de $R_1$ de forma tal que $\left|T(0)\right| = 20 db$
4. (Bonus) Obtener los valores de la red normalizados en frecuencia e impedancia.
5. (Bonus) Calcular las sensibilidades $S^{\omega_0}_C$, $S^{Q}_{R_2}$ y $S^{Q}_{R_3}$.
6. (Bonus) Recalcular los valores de la red para que cumpla con una transferencia Butterworth.
7. (Bonus) Cómo podría obtener un circuito pasabanda con los mismos componentes originales y con qué parámetros quedaría diseñado (Ver ejemplo 4.6 en Schaumann).
8. (Bonus) Simulación circuital de todos los experimentos.


## 1. Deducción de la función de transferencia

Para simplificar el análisis se asumirá que los op-amps se encuentran en equilibrio y su ganancia es arbitrariamente alta. En otras palabras la tensión de entrada diferencial se asumirá que es cero.

Primero, le asignamos nombre a todas las corrientes y tensiones desconocidas en el circuito.

<div>
    <img src="circuito_anotado.png" width="800"/>
</div>

Por ley de nodos, tenemos:

$$I_1 + I_6 = I_2 + I_3$$

Seguimos haciendo los remplazos $I_1 = \frac{V_i}{R_1}$, $I_6 = \frac{V_o}{R_3}$, $I_2 = -\frac{V_a}{R_2}$ y $I_3 = - V_a C s$ para obtener:

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = -V_a\left(\frac{1}{R_2} + C s \right)$$

Remplazando $V_a = I_4 R_3$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = -I_4 R_3\left(\frac{1}{R_2} + C s \right)$$

Remplazando $I_4 = - V_b C s$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = V_b R_3 C s\left(\frac{1}{R_2} + C s \right)$$

Remplazando $V_b = I_5 R_4$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = I_5 R_4 R_3 C s\left(\frac{1}{R_2} + C s \right)$$

Remplazando $I_5 = -\frac{V_o}{R_4}$

$$\frac{V_i}{R_1} + \frac{V_o}{R_3} = - V_o R_3 C s\left(\frac{1}{R_2} + C s \right)$$

A partir de aca se puede reordenar para encontrar $T = \frac{V_o}{V_i}$

$$T(s) = \frac{V_o}{V_i} = - \frac{1}{C^2 R_1 R_3} \frac{1}{s^2 + \frac{1}{C R_2}s + \frac{1}{\left(C R_3\right)^2}}$$

En este punto podemos notar que se trata de un filtro de segundo orden sin ceros, asi que tiene comportamiento de filtro pasa bajos.
Pasamos la funcion de transferencia a su forma estandard en terminos de $\omega_0$, $Q$ y $K$:

$$T(s) = K \frac{\omega_0^2}{s^2 + \frac{\omega_0}{Q}s + \omega_0^2} $$

Donde:

\begin{align*}
 \omega_0 &= \frac{1}{C R_3}  \\
 Q &= \frac{R_2}{R_3} \\
 K &= - \frac{R_3}{R_1}
\end{align*}

### Verificación

Aprovecho para realizar una verificacion utilizando SymPy modificando el codigo del Trabajo Semanal anterior.

In [ ]:
from sympy import solve, symbols, Eq


#Variables conocidas
R1, R2, R3, R4, C, s = symbols("R1 R2 R3 R4 C s")

#Variables desconocidas
H, Vi, Vo, I1, I2, I3, I4, I5, I6, Va, Vb = symbols("H Vi Vo I1 I2 I3 I4 I5 I6 Va Vb")
unknowns = [H, Vi, Vo, I1, I2, I3, I4, I5, I6, Va, Vb]

equations = []

#Ecuaciones de mallas
equations.append(Eq(Vi, I6*R1))
equations.append(Eq(-Va, R2*I1))
equations.append(Eq(-Va, I2*1/(C*s)))
equations.append(Eq(Va, I3*R3))
equations.append(Eq(-Vb, I3*1/(C*s)))
equations.append(Eq(Vb, R4*I4))
equations.append(Eq(-Vo, I4*R4))
equations.append(Eq(Vo, R3*I5))

#Ecuaciones de nodos
equations.append(Eq(I6 + I5, I1 + I2))

#Relacion de H con tensiones
equations.append(Eq(H, Vo/Vi))

#Resolver
sol_H = solve(equations, unknowns, dict=True)[0][H]


sol_H

## 2/3. Elección de componentes

El objetivo propuesto de diseñar el amplificador con parametros $\omega_0 = 1$ y $Q = 3$ sin otra condición nos lleva a inevitablemente elegir un valor para $C$, $R_2$ o $R_3$. Para este diseño propongo $C=1\mathrm{F}$

Esto nos lleva a que:


\begin{align*}
 R_3 &= \frac{1}{C \omega_0} = 1 \Omega  \\
 R_2 &= Q R_3 = 3
\Omega
\end{align*}

Si tambien deseamos cumplir con la condicion de ganancia de continua $\left|T(0)|\right| = 20dB = 10 \times$ debemos evaluar la funcion en $s=0$:

$$|T(0)| = \left|K \frac{\omega_0^2}{0^2 + \frac{\omega_0}{Q} 0^2 + \omega_0^2}\right| = \left|K \omega_0\right| = \left|K\right| = \frac{R_3}{R_1} $$

Lo que buscamos entonces es: $\frac{R_3}{R_1} = 10$. Por lo tanto:

$$R_1 = 0.1 \Omega$$

Finalmente, el valor de $R_4$ en la teoría no afecta el funcionamiento del circuito, pero decido usar $R_4 = 1 \Omega$ para repetir valores.

En resumen:
\begin{align*}
 R_4 &= 0.1 \Omega & R_2 &= 3 \Omega & R_4 = R_3 &= 1 \Omega & C &= 1 F
\end{align*}


## 4. Valores de red normalizados

Para normalizar la red  en frecuencia, utilizamos $\Omega_\omega = \omega_0$

Por ende, la variable compleja de frecuencia normalizada resulta ser: $\$ = \frac{s}{\Omega_\omega} = \frac{s}{\omega_0}$

Y para normalizar en impedancia, utilizaremos $\Omega_z = R_3$. Decidí utilizar (de forma heurística) $R_3$ ya que aparece en la expresión de $\omega_0$ y $Q$, entonces es mas probable de que resulte en mejores simplificaciones de otras variables o componentes.

Los valores de los componentes normalizados resultan:

\begin{align*}
 R_1' &= \frac{R_1}{\Omega_z} = -\frac{1}{K}  \\
 R_2' &= \frac{R_2}{\Omega_z} = \frac{Q R_3}{\Omega_z} = Q \\
 R_3' &= \frac{R_3}{\Omega_z} = 1 \\
 R_4' &= \frac{R_4}{\Omega_z}\\
 C'   &= \Omega_\omega \Omega_z C = \frac{\Omega_\omega \Omega_z}{\omega_0 R_3} = 1
\end{align*}

Y la función de transferencia normalizada en frecuencia e impedancia resulta:

$$ T($) = K \frac{1}{\$^2 + \frac{1}{Q}\$ + 1} $$

Y la representacion gráfica de la red normalizada:

<div>
    <img src="circuito_normalizado.png" width="800"/>
</div>

# 5. Sensibilidades

Partiendo de las expresiones de los parametros en función de los valores de los componentes podemos calcular las sensibilidades pedidas


$S^{\omega_0}_C = \dfrac{C}{\omega_0} \dfrac{\partial \omega_0}{\partial C} = C^2 R_3 \left( - \dfrac{1}{R_3 C^2} \right) = -1$

$S^{Q}_{R_2} = \dfrac{R_2}{Q} \dfrac{\partial Q}{\partial R_2} = R_3 \dfrac{1}{R_3} = 1$

$S^{Q}_{R_3} = \dfrac{R_3}{Q} \dfrac{\partial Q}{\partial R_3} = \dfrac{R_3^2}{R_2} \left(-\dfrac{R_2}{R_3^2}\right) = -1$



# 6. Transferencia Butterworth

Al ser un filtro de segundo orden, solamente debemos encontrar el $Q$ de los unicos polos conjugados del filtro.

Utilizando el metodo de angulos, sabemos que su separacion tiene que ser de $\dfrac{2\pi}{4} = \dfrac{\pi}{2}$ radianes. Entonces la separacion con el eje real de cada polo es de $\varphi = \dfrac{\pi}{4}$ radianes.

A partir de esta informacion, obtenemos $Q$:

$Q = \dfrac{1}{2 cos(\varphi)} = \dfrac{1}{\sqrt{2}}$

Esto nos define los siguientes valores normalizados para los componentes:

\begin{align*}
 R_1' &= -\frac{1}{K}  \\
 R_2' &=  \dfrac{1}{\sqrt{2}} \\
 R_3' &= 1 \\
 R_4' &= 1\\
 C'   &= 1
\end{align*}


# 7. Circuito pasa banda

En su libro, Schaumann muestra la siguiente red:

<div>
    <img src="pass_band_libro.png" width="500"/>
</div>

Se puede ver, como saliendo por la salida de otro de los opamps, se obtiene una respuesta pasa banda.
Agregando definiciones al script de sympy se puede obtener facilmente la expresión de la transferencia pasa banda.

In [ ]:
#Defino la transferencia pasa banda utilizando los simbolos definidos previamente

H_BP = symbols("H_BP")
equations.append(Eq(H_BP, Va/Vi))
unknowns.append(H_BP)

#Despejo H_BP
sol_H = solve(equations, unknowns, dict=True)[0][H_BP]


sol_H

Reordenando, obtenemos:

$T_{BP} = \dfrac{R_2}{R_1} \dfrac{\dfrac{1}{C R_2} s}{s^2 + \dfrac{1}{C R_2}s + \dfrac{1}{C^2 R_3^2}} = K \dfrac{\dfrac{\omega_0}{Q}s}{s^2 + \dfrac{\omega_0}{Q}s + \omega_0^2}$

Por observacion, vemos que las relaciones de los parametros del filtro con los componentes son las siguientes:

\begin{align*}
 \omega_0 &= \frac{1}{C R_3}  \\
 Q &= \frac{R_2}{R_3} \\
 K &= \frac{R_2}{R_1}
\end{align*}

# 8. Simulacion circuital

Utilizando LTSPice podemos realizar una simulacion de las configuraciones pasa bajos y pasa banda de la topologia Ackerberg-Mossberg.

En ambos casos, se eligieron valores de diseño de: $|K| = 1$, $Q=\dfrac{1}{\sqrt{2}}$
Y tambien se simulo utilizando valores desnormalizados para evitar problemas de saturación de la salida de los amplificadores:
$R = 1k\Omega$ y $f_0 = 10kHz$

## Configuracion pasa bajos

### Esquemático

El esquemático de la simulacion es el siguiente:

<div>
    <img src="sim_sch_lp.png" width="400"/>
</div>

### Resultados

Los resultados son los siguientes:

<div>
    <img src="sim_plt_lp.png" width="600"/>
</div>

Se puede observar la caida de -3dB en la frecuencia de corte, el valor esperado para un filtro de maxima planicidad Butterworth



## Configuracion pasa banda

### Esquemático

El esquemático de la simulacion es el siguiente:

<div>
    <img src="sim_sch_bp.png" width="400"/>
</div>

Vale notar que en el caso pasa banda, el valor de la resistencia R1 es dependiende de $Q$ y de $K$ a diferencia de la configuracion pasa bajos en donde solo dependia de $K$

### Resultados

Los resultados son los siguientes:

<div>
    <img src="sim_plt_bp.png" width="600"/>
</div>

La frecuencia central se ubica en 10kHz tal como se diseño y la ganancia en la frecuencia central coincide con lo esperado


# Conclusiones

Este trabajo sirvió como introducción al analisis de filtros, las relaciones entre sus parametros y valores de componentes y como trabajar con valores normalizados. Tambien sirvió como introduccion a una de las estructuras importantes para la implementación de filtros activos, la estructura de Ackerberg-Mossberg y como dependiendo de su uso se pueden obtener dos tipos de respuestas diferentes.